- Need instruction, input, output for the template

In [1]:
import datasets
from copy import deepcopy
dataset = datasets.load_from_disk('./data')
#datasets.load_dataset('b-mc2/sql-create-context',split='train')

orig_dataset = deepcopy(dataset)
dataset
def formatting_prompts_func(sample):
    """Given a sample dictionary with keys "title" and "abstract" format into a prompt.

    Args:
      sample: A sample dictionary from a Hugging Face dataset.

    Returns:
      sample: sample dictionary with "text" key for the formatted prompt.
    """
    #sample['text']=f"[INST] <> Write SQL code to answer the question based on the context. Please wrap your code answer using ```: <> {sample['question']} {sample['context']} [/INST] {sample['answer']}"
    sample['instruction']="You are a helpful assistant who is a SQL expert. Your task is to write a SQL query based on the given context and question."
    sample['input'] = f"""
Context: {sample['sql_context']}
Question: {sample['sql_prompt']}<|end|>
"""
    
    sample['output'] = f"""
SQL Query:
{sample['sql']}
<|end|>
    """
    return sample

def filter_func(example):
    # Replace 'column_name' and 'value' with the actual column name and value
    return (example['sql_complexity'] == 'window functions' 
            or example['sql_complexity'] == 'multiple_joins'
            or example['sql_complexity'] == 'subqueries'
            )
#dataset = load_from_disk('./data')
dataset = (dataset
          # .shuffle(1337)
           #.map(formatting_prompts_func)
           .filter(filter_func)#['train']
            #.train_test_split(test_size=VAL_SIZE)
)
dataset = dataset.map(formatting_prompts_func,
                      remove_columns=['id','domain','domain_description',
                                      'sql_complexity','sql_complexity_description',
                                      'sql_task_type','sql_task_type_description',
                                      'sql_explanation','sql_prompt', 'sql_context', 'sql',])#, batched=True)
dataset = dataset.shuffle(1337).train_test_split(test_size=0.005)


/home/mainuser/anaconda3/envs/sqlft/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Map: 100%|██████████| 13264/13264 [00:00<00:00, 21221.53 examples/s]


In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 13197
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 67
    })
})

In [3]:
dataset.save_to_disk('./data-processed')

Saving the dataset (1/1 shards): 100%|██████████| 67/67 [00:00<00:00, 37589.40 examples/s]
